Use only the data at 2023


In [381]:
import pandas as pd
import numpy as np
import warnings

data_path = "data/"

In [382]:
df = pd.read_csv(f"{data_path}chicago_crime.csv")

In [383]:
df["Date"] = pd.to_datetime(df["Date"])
df_after = df[df["Date"].dt.year.isin([2018, 2019, 2020, 2021, 2022, 2023, 2024])]

In [384]:
df_clean = df_after.copy()
df_clean = df_clean.sort_values(by="Date")
df_clean["Date"] = pd.to_datetime(df_clean["Date"], format="%Y-%m-%dT%H:%M:%S.%f")
df_clean["Year"] = df_clean["Date"].dt.year
df_clean["Month"] = df_clean["Date"].dt.month
df_clean["Day"] = df_clean["Date"].dt.day
df_clean["Hour"] = df_clean["Date"].dt.hour
df_clean["WeekDay"] = df_clean["Date"].dt.weekday
df_clean["IsWeekend"] = (df_clean["WeekDay"] >= 5).astype(bool)
df_clean = df_clean.dropna(subset=["Longitude", "Latitude"])
df_clean["Location Description"] = df_clean["Location Description"].fillna(
    "OTHER (SPECIFY)"
)
hour_bins = [0, 6, 12, 18, 24]
hour_labels = ["Night", "Morning", "Afternoon", "Evening"]
season_bins = [0, 3, 6, 9, 12]
season_labels = ["Winter", "Spring", "Summer", "Fall"]
df_clean["TimeCategory"] = pd.cut(
    df_clean["Hour"], bins=hour_bins, labels=hour_labels, include_lowest=True
)
df_clean["Season"] = pd.cut(
    df_clean["Month"], bins=season_bins, labels=season_labels, include_lowest=True
)

In [385]:
location_mapping = {
    # Residential locations
    "RESIDENTIAL": [
        "APARTMENT",
        "RESIDENCE",
        "RESIDENCE - GARAGE",
        "RESIDENCE - PORCH / HALLWAY",
        "RESIDENCE - YARD (FRONT / BACK)",
        "HOUSE",
        "PORCH",
        "YARD",
        "GARAGE",
        "DRIVEWAY - RESIDENTIAL",
        "DRIVEWAY",
        "CHA APARTMENT",
        "CHA HALLWAY / STAIRWELL / ELEVATOR",
        "CHA HALLWAY",
        "CHA LOBBY",
        "CHA STAIRWELL",
        "CHA GROUNDS",
        "STAIRWELL",
        "BASEMENT",
        "PORCH",
        "HALLWAY",
        "VESTIBULE",
        "GANGWAY",
        "COLLEGE / UNIVERSITY - RESIDENCE HALL",
        "ROOF",
        "CHA ELEVATOR",
        "ELEVATOR",
        "RESIDENTIAL YARD (FRONT/BACK)",
        "RESIDENCE-GARAGE",
        "RESIDENCE PORCH/HALLWAY",
        "CHA HALLWAY/STAIRWELL/ELEVATOR",
        "CHA PARKING LOT/GROUNDS",
        "COLLEGE/UNIVERSITY RESIDENCE HALL",
        "TRAILER",  # Added from warnings
        "CHA PLAY LOT",  # Added from warnings
    ],
    # Commercial retail
    "RETAIL": [
        "DEPARTMENT STORE",
        "SMALL RETAIL STORE",
        "GROCERY FOOD STORE",
        "CONVENIENCE STORE",
        "RETAIL STORE",
        "DRUG STORE",
        "APPLIANCE STORE",
        "PAWN SHOP",
        "AUTO / BOAT / RV DEALERSHIP",
        "LIQUOR STORE",
        "TAVERN / LIQUOR STORE",
        "CLEANING STORE",
        "NEWSSTAND",
        "TAVERN/LIQUOR STORE",
    ],
    # Food and entertainment
    "FOOD_ENTERTAINMENT": [
        "RESTAURANT",
        "BAR OR TAVERN",
        "TAVERN",
        "MOVIE HOUSE / THEATER",
        "SPORTS ARENA / STADIUM",
        "BOWLING ALLEY",
        "POOL ROOM",
        "CASINO/GAMBLING ESTABLISHMENT",
        "ATHLETIC CLUB",
        "CLUB",
        "BANQUET HALL",
        "MOVIE HOUSE/THEATER",
        "SPORTS ARENA/STADIUM",
        "YMCA",  # Added from warnings
    ],
    # Transportation
    "TRANSPORTATION": [
        "CTA TRAIN",
        "CTA PLATFORM",
        "CTA BUS",
        "CTA STATION",
        "CTA BUS STOP",
        "CTA TRACKS - RIGHT OF WAY",
        'CTA "L" TRAIN',
        'CTA "L" PLATFORM',
        "CTA SUBWAY STATION",
        "CTA PROPERTY",
        "CTA PARKING LOT / GARAGE / OTHER PROPERTY",
        "TAXICAB",
        "VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)",
        "VEHICLE NON-COMMERCIAL",
        "VEHICLE - COMMERCIAL",
        "VEHICLE - DELIVERY TRUCK",
        "VEHICLE - COMMERCIAL: ENTERTAINMENT / PARTY BUS",
        "VEHICLE - COMMERCIAL: TROLLEY BUS",
        "OTHER COMMERCIAL TRANSPORTATION",
        "AUTO",
        "OTHER RAILROAD PROPERTY / TRAIN DEPOT",
        "RAILROAD PROPERTY",
        "TRUCK",
        "VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)",
        "CTA GARAGE / OTHER PROPERTY",
        "OTHER RAILROAD PROP / TRAIN DEPOT",
        "VEHICLE-COMMERCIAL",
        "HIGHWAY/EXPRESSWAY",
        "VEHICLE-COMMERCIAL - TROLLEY BUS",  # Added from warnings
        "VEHICLE-COMMERCIAL - ENTERTAINMENT/PARTY BUS",  # Added from warnings
    ],
    # Street and outdoor public areas
    "STREET_OUTDOOR": [
        "STREET",
        "SIDEWALK",
        "ALLEY",
        "VACANT LOT / LAND",
        "VACANT LOT",
        "HIGHWAY / EXPRESSWAY",
        "PARKING LOT",
        "BRIDGE",
        "PARK PROPERTY",
        "LAKEFRONT / WATERFRONT / RIVERBANK",
        "BEACH",
        "FOREST PRESERVE",
        "FARM",
        "RIVER BANK",
        "VACANT LOT/LAND",
        "LAKEFRONT/WATERFRONT/RIVERBANK",
        "LAKE",
        "WOODED AREA",  # Added from warnings
    ],
    # Parking and vehicle related
    "PARKING": [
        "PARKING LOT / GARAGE (NON RESIDENTIAL)",
        "GAS STATION",
        "GAS STATION DRIVE/PROP.",
        "CAR WASH",
        "PARKING LOT",
        "CHA PARKING LOT / GROUNDS",
        "CHA PARKING LOT",
        "POLICE FACILITY / VEHICLE PARKING LOT",
        "AIRPORT PARKING LOT",
        "PARKING LOT/GARAGE(NON.RESID.)",
        "POLICE FACILITY/VEH PARKING LOT",
        "GARAGE/AUTO REPAIR",  # Added from warnings
    ],
    # Financial institutions
    "FINANCIAL": [
        "BANK",
        "CURRENCY EXCHANGE",
        "ATM (AUTOMATIC TELLER MACHINE)",
        "CREDIT UNION",
        "SAVINGS AND LOAN",
    ],
    # Office and business
    "OFFICE_BUSINESS": [
        "COMMERCIAL / BUSINESS OFFICE",
        "MEDICAL / DENTAL OFFICE",
        "WAREHOUSE",
        "FACTORY / MANUFACTURING BUILDING",
        "OFFICE",
        "CONSTRUCTION SITE",
        "ANIMAL HOSPITAL",
        "FACTORY/MANUFACTURING BUILDING",
        "MEDICAL/DENTAL OFFICE",
    ],
    # Educational
    "EDUCATIONAL": [
        "SCHOOL - PUBLIC BUILDING",
        "SCHOOL - PUBLIC GROUNDS",
        "SCHOOL - PRIVATE BUILDING",
        "SCHOOL - PRIVATE GROUNDS",
        "COLLEGE / UNIVERSITY - GROUNDS",
        "LIBRARY",
        "SCHOOL YARD",
        "DAY CARE CENTER",
        "PUBLIC GRAMMAR SCHOOL",
        "SCHOOL, PUBLIC, BUILDING",
        "SCHOOL, PUBLIC, GROUNDS",
        "SCHOOL, PRIVATE, BUILDING",
        "SCHOOL, PRIVATE, GROUNDS",
        "COLLEGE/UNIVERSITY GROUNDS",
    ],
    # Government and public services
    "GOVERNMENT": [
        "GOVERNMENT BUILDING / PROPERTY",
        "FEDERAL BUILDING",
        "HOSPITAL BUILDING / GROUNDS",
        "HOSPITAL",
        "FIRE STATION",
        "POLICE FACILITY / VEHICLE PARKING LOT",
        "NURSING / RETIREMENT HOME",
        "JAIL / LOCK-UP FACILITY",
        "POLICE FACILITY",
        "GOVERNMENT BUILDING/PROPERTY",
        "HOSPITAL BUILDING/GROUNDS",
        "NURSING HOME/RETIREMENT HOME",
        "NURSING HOME",
        "GOVERNMENT BUILDING",  # Added from warnings
    ],
    "OTHER": [
        "HOTEL / MOTEL",
        "HOTEL",
        "MOTEL",
        "HOTEL/MOTEL",
        "CHURCH / SYNAGOGUE / PLACE OF WORSHIP",
        "CHURCH/SYNAGOGUE/PLACE OF WORSHIP",
        "BARBERSHOP",
        "BARBER SHOP/BEAUTY SALON",
        "KENNEL",
        "HORSE STABLE",  # Added from warnings
        "OTHER",
        "ABANDONED BUILDING",
        "CEMETARY",
        "BOAT / WATERCRAFT",
        "BOAT/WATERCRAFT",
        "OTHER (SPECIFY)",
        "AIRPORT TERMINAL LOWER LEVEL - SECURE AREA",
        "AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA",
        "AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA",
        "AIRPORT TERMINAL UPPER LEVEL - SECURE AREA",
        "AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA",
        "AIRPORT BUILDING NON-TERMINAL - SECURE AREA",
        "AIRPORT EXTERIOR - SECURE AREA",
        "AIRPORT EXTERIOR - NON-SECURE AREA",
        "AIRPORT TRANSPORTATION SYSTEM (ATS)",
        "AIRPORT VENDING ESTABLISHMENT",
        "AIRPORT/AIRCRAFT",
        "AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA",
        "AIRCRAFT",
        "COIN OPERATED MACHINE",
    ],
}
reverse_mapping = {}
for group, locations in location_mapping.items():
    for location in locations:
        reverse_mapping[location] = group


def map_to_location_group(location):
    if location in reverse_mapping:
        return reverse_mapping[location]
    if location != "OTHER (SPECIFY)":
        warnings.warn("Unknown location:" + location)
    return "OTHER"


df_clean["Location Group"] = df_clean["Location Description"].apply(
    map_to_location_group
)
df_clean["Location Group"] = df_clean["Location Description"].apply(
    map_to_location_group
)

In [386]:
from pandas.tseries.holiday import USFederalHolidayCalendar

cal = USFederalHolidayCalendar()

start_date = df_clean["Date"].min()
end_date = df_clean["Date"].max()

# Get holidays between these dates
holidays = cal.holidays(start=start_date, end=end_date)

# Check if each date is a holiday
df_clean["IsHoliday"] = df_clean["Date"].dt.date.isin(holidays.date).astype(int)

In [387]:
df_analysis = pd.DataFrame(
    {
        "Column": df_clean.columns,
        "Unique Values": df_clean.nunique().values,
        "Type": df_clean.dtypes.values,
        "Is Null": df_clean.isnull().sum().values,
    }
)
df_analysis.sort_values("Unique Values", ascending=False)

,Column,Unique Values,Type,Is Null
0,ID,1687016,int64,0
1,Case Number,1686809,object,0
2,Date,825170,datetime64[ns],0
21,Location,384399,object,0
19,Latitude,384160,float64,0
20,Longitude,384026,float64,0
16,Y Coordinate,113637,float64,0
15,X Coordinate,68706,float64,0
3,Block,37264,object,0
18,Updated On,4320,object,0


# Classification Dataset


In [388]:
# First create your year-based split
df_train = df_clean[df_clean["Year"].isin([2019, 2020, 2021, 2022, 2023])].copy()
df_test = df_clean[df_clean["Year"] == 2024].copy()
impute_values = {
    "Ward": df_train["Ward"].mode()[0],
    "Community Area": df_train["Community Area"].mode()[0],
}
for column, value in impute_values.items():
    df_train[column] = df_train[column].fillna(value)
    df_test[column] = df_test[column].fillna(value)

In [389]:
df_train_sorted_lat = df_train["Latitude"].sort_values()
lat_bin_edges = np.linspace(
    df_train_sorted_lat.min(), df_train_sorted_lat.max(), 51
)  # 51 edges for 50 bins
df_train["lat_bin"] = pd.cut(
    df_train["Latitude"], bins=lat_bin_edges, include_lowest=True
)
df_test["lat_bin"] = pd.cut(
    df_test["Latitude"], bins=lat_bin_edges, include_lowest=True
)
df_train_sorted_lon = df_train["Longitude"].sort_values()
lon_bin_edges = np.linspace(
    df_train_sorted_lon.min(), df_train_sorted_lon.max(), 51
)  # 51 edges for 50 bins
df_train["lon_bin"] = pd.cut(
    df_train["Longitude"], bins=lon_bin_edges, include_lowest=True
)
df_test["lon_bin"] = pd.cut(
    df_test["Longitude"], bins=lon_bin_edges, include_lowest=True
)

In [390]:
location_counts = df_train["Location Description"].value_counts()
threshold = round(
    len(df_train) * 0.01
)  # Keep categories that appear in at least 1% of data
print(threshold)
keep_locations = location_counts[location_counts >= threshold].index


def map_rare_categories(df, column, keep_values):
    df_copy = df.copy()
    df_copy.loc[~df_copy[column].isin(keep_values), column] = "Other"
    return df_copy


df_train = map_rare_categories(df_train, "Location Description", keep_locations)
df_test = map_rare_categories(df_test, "Location Description", keep_locations)
df_train.to_csv(f"{data_path}classification_train_data.csv", index=False)
df_test.to_csv(f"{data_path}classification_test_data.csv", index=False)

11661


In [391]:
df_train.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Day,Hour,WeekDay,IsWeekend,TimeCategory,Season,Location Group,IsHoliday,lat_bin,lon_bin
7031276,11737856,JC323472,2019-01-01,020XX W 21ST PL,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,APARTMENT,False,True,...,1,0,1,False,Night,Winter,RESIDENTIAL,1,"(41.806, 41.915]","(-87.691, -87.608]"
6901360,11563351,JC112682,2019-01-01,045XX S EVANS AVE,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,APARTMENT,False,True,...,1,0,1,False,Night,Winter,RESIDENTIAL,1,"(41.806, 41.915]","(-87.691, -87.608]"
7106888,11803895,JC404008,2019-01-01,070XX S CALUMET AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,1,0,1,False,Night,Winter,RESIDENTIAL,1,"(41.698, 41.806]","(-87.691, -87.608]"
7893135,13187275,JG387404,2019-01-01,030XX W MARQUETTE RD,0263,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - KNIFE / CUTTING INSTRUMENT,RESIDENCE,False,False,...,1,0,1,False,Night,Winter,RESIDENTIAL,1,"(41.698, 41.806]","(-87.774, -87.691]"
6998426,11610512,JC170353,2019-01-01,043XX S HOMAN AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,...,1,0,1,False,Night,Winter,RESIDENTIAL,1,"(41.806, 41.915]","(-87.774, -87.691]"


In [392]:
df_test.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Day,Hour,WeekDay,IsWeekend,TimeCategory,Season,Location Group,IsHoliday,lat_bin,lon_bin
7993926,13324885,JH100099,2024-01-01,034XX W DRUMMOND PL,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,...,1,0,0,False,Night,Winter,RESIDENTIAL,1,"(41.915, 42.023]","(-87.774, -87.691]"
8000033,13326163,JH101560,2024-01-01,040XX S DREXEL BLVD,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,1,0,0,False,Night,Winter,STREET_OUTDOOR,1,"(41.806, 41.915]","(-87.608, -87.525]"
8093930,13471561,JH277375,2024-01-01,031XX S HALSTED ST,0810,THEFT,OVER $500,Other,False,False,...,1,0,0,False,Night,Winter,GOVERNMENT,1,"(41.806, 41.915]","(-87.691, -87.608]"
7998874,13325240,JH100545,2024-01-01,081XX S ESSEX AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,1,0,0,False,Night,Winter,STREET_OUTDOOR,1,"(41.698, 41.806]","(-87.608, -87.525]"
7996615,13326167,JH101437,2024-01-01,040XX S DREXEL BLVD,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,1,0,0,False,Night,Winter,STREET_OUTDOOR,1,"(41.806, 41.915]","(-87.608, -87.525]"


In [393]:
df_train.shape, df_test.shape

((1166140, 33), (257320, 33))

# Time-series Dataset


In [ ]:
df_time = df_clean.copy()
df_time.head()
holiday_calendar = (
    df_time[df_time["IsHoliday"] == 1]
    .drop_duplicates(subset=["Year", "Month", "Day"])  # count each holiday once per day
    .groupby(["Year", "Month"], as_index=False)
    .agg(holiday_count=("Day", "count"))
)
df_time["crime_count"] = 1
crime_summary = df_time.groupby(
    ["Location Group", "District", "Year", "Month"], as_index=False
).agg(crime_count=("crime_count", "sum"))
df_time = pd.merge(crime_summary, holiday_calendar, on=["Year", "Month"], how="left")
df_time["holiday_count"] = df_time["holiday_count"].fillna(0).astype(int)
df_time

,Location Group,District,Year,Month,crime_count,holiday_count
0,EDUCATIONAL,1.0,2018,1,28,2
1,EDUCATIONAL,1.0,2018,2,25,1
2,EDUCATIONAL,1.0,2018,3,26,0
3,EDUCATIONAL,1.0,2018,4,31,0
4,EDUCATIONAL,1.0,2018,5,28,1
...,...,...,...,...,...,...
20220,TRANSPORTATION,25.0,2024,9,32,1
20221,TRANSPORTATION,25.0,2024,10,30,1
20222,TRANSPORTATION,25.0,2024,11,35,2
20223,TRANSPORTATION,25.0,2024,12,26,1


In [ ]:
df_time["sin_month"] = np.sin(2 * np.pi * (df_time["Month"] - 1) / 12)
df_time["cos_month"] = np.cos(2 * np.pi * (df_time["Month"] - 1) / 12)
# Create a time ID variable for easier shifting
df_time["time_id"] = df_time["Year"] * 12 + df_time["Month"]
for location_group, location_df in df_time.groupby(["Location Group"]):
    # Get indices for this location group
    indices = location_df.index

    # Make sure we're still working with time-ordered data
    temp_df = df_time.loc[indices].sort_values("time_id")

    # Add lag features
    df_time.loc[indices, "crime_count_lag1"] = temp_df["crime_count"].shift(1)
    df_time.loc[indices, "crime_count_lag2"] = temp_df["crime_count"].shift(2)
    df_time.loc[indices, "crime_count_lag3"] = temp_df["crime_count"].shift(3)
    df_time.loc[indices, "crime_count_lag6"] = temp_df["crime_count"].shift(6)

    # Add 12-month lag (same month previous year)
    df_time.loc[indices, "crime_count_lag12"] = temp_df["crime_count"].shift(12)

    # Moving averages
    df_time.loc[indices, "crime_count_ma3"] = (
        temp_df["crime_count"].rolling(window=3).mean().shift(1)
    )
    df_time.loc[indices, "crime_count_ma6"] = (
        temp_df["crime_count"].rolling(window=6).mean().shift(1)
    )
lag_columns = [
    "crime_count_lag1",
    "crime_count_lag2",
    "crime_count_lag3",
    "crime_count_lag6",
    "crime_count_lag12",
    "crime_count_ma3",
    "crime_count_ma6",
]

df_time.drop("time_id", axis=1, inplace=True)

In [396]:
unique_years = df_clean["Year"].unique()
unique_months = range(1, 13)  # Months 1-12
unique_locations = df_clean["Location Group"].unique()
unique_districts = df_clean["District"].unique()

# Create all possible combinations
from itertools import product

all_combinations = list(
    product(unique_years, unique_months, unique_locations, unique_districts)
)
complete_grid = pd.DataFrame(
    all_combinations, columns=["Year", "Month", "Location Group", "District"]
)
df_time_complete = pd.merge(
    complete_grid,
    df_time,
    on=["Year", "Month", "Location Group", "District"],
    how="left",
)

df_time_complete["crime_count"] = df_time_complete["crime_count"].fillna(0)
df_time_complete["sin_month"] = np.sin(2 * np.pi * (df_time_complete["Month"] - 1) / 12)
df_time_complete["cos_month"] = np.cos(2 * np.pi * (df_time_complete["Month"] - 1) / 12)
df_time_complete.describe()

,Year,Month,District,crime_count,holiday_count,sin_month,cos_month,crime_count_lag1,crime_count_lag2,crime_count_lag3,crime_count_lag6,crime_count_lag12,crime_count_ma3,crime_count_ma6
count,21252.000000,21252.000000,21252.000000,21252.000000,20225.000000,2.125200e+04,2.125200e+04,20214.000000,20203.000000,20192.000000,20159.000000,20093.000000,20192.000000,20159.000000
mean,2021.000000,6.500000,13.000000,79.381517,0.881088,4.279572e-17,-6.954305e-17,83.425843,83.447557,83.451813,83.476859,83.494998,83.434033,83.455256
std,2.000047,3.452134,7.967512,129.615140,0.661999,7.071234e-01,7.071234e-01,131.577222,131.600082,131.608204,131.674817,131.712929,122.374198,120.128871
min,2018.000000,1.000000,1.000000,0.000000,0.000000,-1.000000e+00,-1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.333333
25%,2019.000000,3.750000,6.000000,9.000000,0.000000,-5.915064e-01,-5.915064e-01,11.000000,11.000000,11.000000,11.000000,11.000000,13.000000,13.500000
50%,2021.000000,6.500000,12.000000,24.000000,1.000000,6.123234e-17,-6.123234e-17,25.000000,25.000000,25.000000,25.000000,25.000000,28.666667,30.500000
75%,2023.000000,9.250000,19.000000,63.000000,1.000000,5.915064e-01,5.915064e-01,69.750000,70.000000,70.000000,69.500000,70.000000,66.000000,63.000000
max,2024.000000,12.000000,31.000000,989.000000,2.000000,1.000000e+00,1.000000e+00,989.000000,989.000000,989.000000,989.000000,989.000000,679.000000,520.333333


In [397]:
df_time_complete.sort_values(
    by=["Year", "Month", "Location Group", "District"],
    inplace=True,
)
df_time_complete

,Year,Month,Location Group,District,crime_count,holiday_count,sin_month,cos_month,crime_count_lag1,crime_count_lag2,crime_count_lag3,crime_count_lag6,crime_count_lag12,crime_count_ma3,crime_count_ma6
171,2018,1,EDUCATIONAL,1.0,28.0,2.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,2018,1,EDUCATIONAL,2.0,25.0,2.0,0.0,1.000000,37.0,11.0,8.0,8.0,24.0,18.666667,14.500000
180,2018,1,EDUCATIONAL,3.0,21.0,2.0,0.0,1.000000,28.0,NaN,NaN,NaN,NaN,NaN,NaN
163,2018,1,EDUCATIONAL,4.0,30.0,2.0,0.0,1.000000,21.0,28.0,NaN,NaN,NaN,NaN,NaN
172,2018,1,EDUCATIONAL,5.0,18.0,2.0,0.0,1.000000,30.0,21.0,28.0,NaN,NaN,26.333333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21227,2024,12,TRANSPORTATION,20.0,8.0,1.0,-0.5,0.866025,28.0,95.0,8.0,27.0,30.0,43.666667,34.333333
21212,2024,12,TRANSPORTATION,22.0,8.0,1.0,-0.5,0.866025,45.0,8.0,28.0,24.0,22.0,27.000000,34.666667
21206,2024,12,TRANSPORTATION,24.0,25.0,1.0,-0.5,0.866025,23.0,41.0,38.0,45.0,24.0,34.000000,30.000000
21207,2024,12,TRANSPORTATION,25.0,26.0,1.0,-0.5,0.866025,34.0,20.0,26.0,25.0,45.0,26.666667,25.500000


In [398]:
df_time_analysis = pd.DataFrame(
    {
        "Column": df_time_complete.columns,
        "Unique Values": df_time_complete.nunique().values,
        "Type": df_time_complete.dtypes.values,
        "Is Null": df_time_complete.isnull().sum().values,
    }
)
df_time_analysis

,Column,Unique Values,Type,Is Null
0,Year,7,int32,0
1,Month,12,int64,0
2,Location Group,11,object,0
3,District,23,float64,0
4,crime_count,663,float64,0
5,holiday_count,3,float64,1027
6,sin_month,11,float64,0
7,cos_month,11,float64,0
8,crime_count_lag1,662,float64,1038
9,crime_count_lag2,662,float64,1049


In [399]:
df_time_train = df_time[df_time["Year"].isin([2019, 2020, 2021, 2022, 2023])].copy()
df_time_test = df_time[df_time["Year"] == 2024].copy()
df_time_train.to_csv(f"{data_path}time_series_train_data.csv", index=False)
df_time_test.to_csv(f"{data_path}time_series_test_data.csv", index=False)

In [403]:
rnn_full = df_time_complete.drop(
    [
        "crime_count_lag1",
        "crime_count_lag2",
        "crime_count_lag3",
        "crime_count_lag6",
        "crime_count_lag12",
        "crime_count_ma3",
        "crime_count_ma6",
    ],
    axis=1,
)
rnn_features = [
    "Year",
    "Month",
    "District",
    "Location Group",
    "holiday_count",
    "sin_month",
    "cos_month",
]
rnn_full = rnn_full.groupby(rnn_features).agg({"crime_count": "sum"}).reset_index()
rnn_full.to_csv(f"{data_path}rnn_full_data.csv", index=False)